In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Read data

In [ ]:
import pandas as pd
import numpy as np

sales_train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
item_categories = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
sample_submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')

In [ ]:
sales_train.head()

# 2. Data Preprocessing

Using Oct, 2015 as importance feature to predict Nov, 2015 sold.<br>
date_block_num of Oct,2015 is 33.

In [ ]:
sales_train[sales_train['date_block_num'] == 33].sort_values(by='item_cnt_day', ascending=False)

Something wrong with first row(2909818). Let check it

In [ ]:
sales_train[sales_train['item_id'] == 11373].sort_values(['item_price'])

In [ ]:
sales_train['item_price'][2909818] = np.nan
sales_train['item_cnt_day'][2909818] = np.nan

sales_train['item_price'][2909818] = sales_train[
    (sales_train['shop_id'] ==12) & 
    (sales_train['item_id'] == 11373) &
    (sales_train['date_block_num'] == 33)]['item_price'].median()

sales_train['item_cnt_day'][2909818] = round(sales_train[(sales_train['shop_id'] ==12) &
                                                         (sales_train['item_id'] == 11373) &
                                                         (sales_train['date_block_num'] == 33)]['item_cnt_day'].median())

Almost item_id 11373 sold by shop_id 12. I will check that shop ~~

In [ ]:
sales_train[sales_train['shop_id'] == 12].sort_values(by='item_price', ascending=False)

In [ ]:
sales_train[sales_train['item_id'] == 6066].sort_values(['item_price'])

In [ ]:
sales_train[sales_train['item_id'] == 11365].sort_values(by='item_price', ascending=False)

In [ ]:
sales_train['item_price'][885138] = np.nan
sales_train['item_price'][885138] = sales_train[(sales_train['item_id'] == 11365) &
                                              (sales_train['shop_id'] ==12) &
                                              (sales_train['date_block_num'] == 8)]['item_price'].median()

## Remove outliers

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,4))
plt.xlim(-100, 3000)

flierprops = dict(marker='o', markerfacecolor='purple', markersize=6,
                  linestyle='none', markeredgecolor='black')
sns.boxplot(x=sales_train.item_cnt_day, flierprops=flierprops)

plt.figure(figsize=(10,4))
plt.xlim(sales_train.item_price.min(), sales_train.item_price.max()*1.1)

sns.boxplot(x=sales_train.item_price, flierprops=flierprops)

In [ ]:
sales_train = sales_train[(sales_train.item_price < 300000 )& (sales_train.item_cnt_day < 1000)]

In [ ]:
sales_train = sales_train[sales_train.item_price > 0].reset_index(drop = True)
sales_train.loc[sales_train.item_cnt_day < 1, "item_cnt_day"] = 0

In [ ]:
sales_train.loc[sales_train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57

sales_train.loc[sales_train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58

sales_train.loc[sales_train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

## Droping all the shop are in sales_train but are not in test

In [ ]:
in_train = np.sort(sales_train['shop_id'].unique())
in_test = np.sort(test['shop_id'].unique())

print('list shop_id are in train set but are not in test set')
for i in in_train:
    if i not in in_test:
        print(i, end=', ')

In [ ]:
print('Before:', sales_train.shape)

In [ ]:
sales_train = sales_train.merge(test[['shop_id']].drop_duplicates(), how='inner')

sales_train['date'] = pd.to_datetime(sales_train['date'], format = '%d.%m.%Y')

In [ ]:
print('After:', sales_train.shape)

### Aggregate data

For every month we create a grid from all shops/items combinations from that month

In [ ]:
from itertools import product

grid = []
for block_num in sales_train['date_block_num'].unique():
    cur_shops = sales_train[sales_train['date_block_num'] == block_num]['shop_id'].unique()
    cur_items = sales_train[sales_train['date_block_num'] == block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

idx_columns = ['shop_id', 'item_id', 'date_block_num']
grid = pd.DataFrame(np.vstack(grid), columns = idx_columns, dtype = np.int32)
grid.shape

in Evalution :"*Submissions are evaluated by root mean squared error (RMSE). True target values are clipped into [0,20] range."* So, clip\[0,20\] in train and test set

In [ ]:
sales_train['item_cnt_day'] = sales_train['item_cnt_day'].clip(0,20)

gb_cnt = sales_train.groupby(idx_columns)['item_cnt_day'].agg(['sum']).reset_index().rename(columns = {'sum': 'item_cnt_month'})
gb_cnt['item_cnt_month'] = gb_cnt['item_cnt_month'].clip(0,20).astype(np.int)

join aggregated data to the grid

In [ ]:
train = pd.merge(grid, gb_cnt, how='left', on=idx_columns).fillna(0)
train['item_cnt_month'] = train['item_cnt_month'].astype(int)

In [ ]:
train.info()

In [ ]:
cur_dtypes = [i for i in train if train[i].dtype in ['int32', 'int64']]
train[cur_dtypes] = train[cur_dtypes].astype(np.int16)
train.info()

Sort train data

In [ ]:
train.sort_values(['date_block_num','shop_id','item_id'], inplace=True)
train.head()

In [ ]:
print(sales_train['item_cnt_day'].sum())
print(train['item_cnt_month'].sum())
print(gb_cnt['item_cnt_month'].sum())

In [ ]:
train = train.merge(items[['item_id', 'item_category_id']], on = ['item_id'], how = 'left')
test = test.merge(items[['item_id', 'item_category_id']], on = ['item_id'], how = 'left')

### Encode item_category_name

I using item_categories-translated by [**deargle**](https://www.kaggle.com/c/competitive-data-science-predict-future-sales/discussion/54949)

In [ ]:
categ_translated = pd.read_csv('../input/predict-future-sales-english/item_categories-translated.csv')

item_categories = item_categories.merge(categ_translated[['item_category_name_translated', 'item_category_id']], on=['item_category_id'], how='left')

In [ ]:
for idx, name in enumerate(item_categories.item_category_name_translated):
    print(idx, name)

In [ ]:
list_categ = list(item_categories.item_category_name_translated)

for i in range(1, 8):
    list_categ[i] = 'Accessories'
for i in range(10, 18):
    list_categ[i] = 'Game Consoles'
for i in range(18, 25):
    list_categ[i] = 'Games'
list_categ[25] = 'Accessories for games'
for i in range(26, 28):
    list_categ[i] = 'Phone Games'
for i in range(28, 32):
    list_categ[i] = 'PC Games'
for i in range(32, 37):
    list_categ[i] = 'Cards'
for i in range(37, 42):
    list_categ[i] = 'Cinema'
for i in range(42, 55):
    list_categ[i] = 'Books'
for i in range(55, 61):
    list_categ[i] = 'Music'
for i in range(61, 73):
    list_categ[i] = 'Gifts'
for i in range(73, 79):
    list_categ[i] = 'Programs'
for i in range(79, 81):
    list_categ[i] = 'Office'
for i in range(81, 83):
    list_categ[i] = 'Clean'

In [ ]:
from sklearn import preprocessing

lb_encode = preprocessing.LabelEncoder()
item_categories['item_category_id_fixed'] = lb_encode.fit_transform(list_categ)

train = train.merge(item_categories[['item_category_id_fixed', 'item_category_id']], on = ['item_category_id'], how = 'left')

test = test.merge(item_categories[['item_category_id_fixed', 'item_category_id']], on = ['item_category_id'], how = 'left')

In [ ]:
train.head()

Add item/shop pair mean-encodings

In [ ]:
from tqdm import tqdm
from sklearn.model_selection import KFold

global_mean =  train['item_cnt_month'].mean()
y_train = train['item_cnt_month'].values

mean_encoded_col = ['shop_id', 'item_id', 'item_category_id', 'item_category_id_fixed']

for col in tqdm(mean_encoded_col):
    col_train = train[[col] + ['item_cnt_month']]
    corr_coefs = pd.DataFrame(columns = ['Cor'])
    # Mean encoding KFold scheme
    kf = KFold(n_splits = 5, shuffle = False, random_state = 0)
    
    col_train[col + '_cnt_month_mean_KFold'] = global_mean
    for idx_train, idx_val in kf.split(col_train):
        X_train, X_val = col_train.iloc[idx_train], col_train.iloc[idx_val]
        means = X_val[col].map(X_train.groupby(by=col)['item_cnt_month'].mean())
        X_val[col + '_cnt_month_mean_KFold'] = means
        col_train.iloc[idx_val] = X_val
        print(X_val.head(5))
    col_train.fillna(global_mean, inplace=True)
    corr_coefs.loc[col + '_cnt_month_mean_KFold'] = np.corrcoef(y_train, col_train[col + '_cnt_month_mean_KFold'])[0][1]
    
    # Mean encodings - Leave-one-out scheme
    item_id_target_sum = col_train.groupby(col)['item_cnt_month'].sum()
    item_id_target_count = col_train.groupby(col)['item_cnt_month'].count()
    col_train[col + '_cnt_month_sum'] = col_train[col].map(item_id_target_sum)
    col_train[col + '_cnt_month_count'] = col_train[col].map(item_id_target_count)
    col_train[col + '_target_mean_LOO'] = (col_train[col + '_cnt_month_sum'] - col_train['item_cnt_month']) / (col_train[col + '_cnt_month_count'] - 1)
    col_train.fillna(global_mean, inplace = True)
    corr_coefs.loc[col + '_target_mean_LOO'] = np.corrcoef(y_train, col_train[col + '_target_mean_LOO'])[0][1]
    
    # Mean encodings - Smoothing
    item_id_target_mean = col_train.groupby(col)['item_cnt_month'].mean()
    item_id_target_count = col_train.groupby(col)['item_cnt_month'].count()
    col_train[col + '_cnt_month_mean'] = col_train[col].map(item_id_target_mean)
    col_train[col + '_cnt_month_count'] = col_train[col].map(item_id_target_count)
    alpha = 100
    col_train[col + '_cnt_month_mean_Smooth'] = (col_train[col + '_cnt_month_mean'] *  col_train[col + '_cnt_month_count'] + global_mean * alpha) / (alpha + col_train[col + '_cnt_month_count'])
    col_train[col + '_cnt_month_mean_Smooth'].fillna(global_mean, inplace=True)
    corr_coefs.loc[col + '_cnt_month_mean_Smooth'] = np.corrcoef(y_train, col_train[col + '_cnt_month_mean_Smooth'])[0][1]
    
    # 3.1.4 Mean encodings - Expanding mean scheme
    cumsum = col_train.groupby(col)['item_cnt_month'].cumsum() - col_train['item_cnt_month']
    sumcnt = col_train.groupby(col).cumcount()
    col_train[col + '_cnt_month_mean_Expanding'] = cumsum / sumcnt
    col_train[col + '_cnt_month_mean_Expanding'].fillna(global_mean, inplace=True)
    corr_coefs.loc[col + '_cnt_month_mean_Expanding'] = np.corrcoef(y_train, col_train[col + '_cnt_month_mean_Expanding'])[0][1]
    train = pd.concat([train, col_train[corr_coefs['Cor'].idxmax()]], axis = 1)
    print(corr_coefs.sort_values('Cor'))

### Feature Engineering 

In [ ]:
# Nov, 2015
test['date_block_num'] = 34

all_data = pd.concat([train, test], axis = 0)
all_data = all_data.drop(columns = ['ID'])

In [ ]:
float_col = [f for f in all_data if all_data[f].dtype == 'float64']
int_col = [i for i in all_data if all_data[i].dtype == 'int64']

all_data[float_col] = all_data[float_col].astype(np.float32)
all_data[int_col] = all_data[int_col].astype(np.int16)

In [ ]:
index_cols = ['shop_id', 'item_id', 'item_category_id', 'item_category_id_fixed', 'date_block_num']
cols_to_rename = list(all_data.columns.difference(index_cols))

shift_range = [1, 2, 3, 4, 12]
for month_shift in tqdm(shift_range):
    train_shift = all_data[index_cols + cols_to_rename].copy()
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)
    
    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)
del train_shift

In [ ]:
# Don't use old data from year 2013
all_data = all_data[all_data['date_block_num'] >= 12]

lag_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]]

float_col = [f for f in all_data if all_data[f].dtype == 'float64']
int_col = [i for i in all_data if all_data[i].dtype in ['int64', 'int32']]

all_data[float_col] = all_data[float_col].astype(np.float32)
all_data[int_col] = all_data[int_col].astype(np.int16)

Creating date feature

In [ ]:
dates_train = sales_train[['date', 'date_block_num']].drop_duplicates()
dates_test = dates_train[dates_train['date_block_num'] == 34-12]
dates_test['date_block_num'] = 34
dates_test['date'] = dates_test['date'] + pd.DateOffset(years=1)
dates_all = pd.concat([dates_train, dates_test])


dates_all['dow'] = dates_all['date'].dt.dayofweek
dates_all['year'] = dates_all['date'].dt.year
dates_all['month'] = dates_all['date'].dt.month

dates_all = pd.get_dummies(dates_all, columns=['dow'])

dow_col = ['dow_' + str(x) for x in range(7)]

date_features = dates_all.groupby(['year', 'month', 'date_block_num'])[dow_col].agg('sum').reset_index()
date_features['days_of_month'] = date_features[dow_col].sum(axis=1)
date_features['year'] = date_features['year'] - 2013
date_features = date_features[['month', 'year', 'days_of_month', 'date_block_num']]

all_data = all_data.merge(date_features, on = 'date_block_num', how = 'left')

date_columns = date_features.columns.difference(set(index_cols))

Scale feature columns

In [ ]:
from sklearn.preprocessing import StandardScaler

train = all_data[all_data['date_block_num']!= all_data['date_block_num'].max()]
test = all_data[all_data['date_block_num']== all_data['date_block_num'].max()]
sc = StandardScaler()

to_drop_cols = ['date_block_num']
feature_columns = list(set(lag_cols + index_cols + list(date_columns)).difference(to_drop_cols))

train[feature_columns] = sc.fit_transform(train[feature_columns])

test[feature_columns] = sc.transform(test[feature_columns])

all_data = pd.concat([train, test], axis = 0)

In [ ]:
float_col = [f for f in all_data if all_data[f].dtype == 'float64']
int_col = [i for i in all_data if all_data[i].dtype in ['int64', 'int32']]

all_data[float_col] = all_data[float_col].astype(np.float32)
all_data[int_col] = all_data[int_col].astype(np.int16)

# 3. Model

In [ ]:
dates = all_data['date_block_num']
print('Test `date_block_num` is %d' % dates.max())
print(len(feature_columns))

In [ ]:
all_data.info()

In [ ]:
all_data[all_data["date_block_num"]==34].shape

In [ ]:
X_test.shape

In [ ]:
X_train = all_data[all_data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = all_data[all_data.date_block_num < 33]['item_cnt_month']
X_valid = all_data[all_data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = all_data[all_data.date_block_num == 33]['item_cnt_month']
X_test = all_data[all_data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [ ]:
Y_train = Y_train.clip(0, 20)
Y_valid = Y_valid.clip(0, 20)

In [ ]:
from xgboost import XGBRegressor
from matplotlib.pylab import rcParams

model = XGBRegressor(max_depth=10, n_estimators=1000, min_child_weight=0.5, colsample_bytree=0.8, subsample=0.8, eta=0.1, seed=42)

model.fit(X_train, Y_train, eval_metric="rmse", eval_set=[(X_train, Y_train), (X_valid, Y_valid)], verbose=True, early_stopping_rounds = 20)

In [ ]:
test_1 = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(X_test).clip(0, 20)

submission = pd.DataFrame({'ID': test_1.index, 'item_cnt_month': Y_test})
submission.to_csv('xgb_submission.csv', index=False)

In [ ]:
from xgboost import plot_importance

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

plot_features(model, (10,14))

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
scoringMethod = 'r2'
num_first_level_models = 3

# Train meta-features M = 15 (12 + 15 = 27)
months_to_generate_meta_features = range(27,last_block +1)
mask = dates.isin(months_to_generate_meta_features)

target = 'item_cnt_month'

y_all_level2 = all_data[target][mask].values

X_all_level2 = np.zeros([y_all_level2.shape[0], num_first_level_models])

In [ ]:
from sklearn.linear_model import LinearRegression, SGDRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

slice_start = 0
Target = 'item_cnt_month'

for cur_block_num in tqdm(months_to_generate_meta_features):
    print('-' * 50)
    print('Start training for month%d'% cur_block_num)

    cur_X_train = all_data.loc[dates <  cur_block_num][feature_columns]
    cur_X_test =  all_data.loc[dates == cur_block_num][feature_columns]

    cur_y_train = all_data.loc[dates <  cur_block_num, Target].values
    cur_y_test =  all_data.loc[dates == cur_block_num, Target].values

    # Create Numpy arrays of train, test and target dataframes to feed into models
    train_x = cur_X_train.values
    train_y = cur_y_train.ravel()
    test_x = cur_X_test.values
    test_y = cur_y_test.ravel()

    preds = []

    sgdr= SGDRegressor(penalty = 'l2', random_state = 0 )
    lgb_params = {'feature_fraction': 0.75, 'metric': 'rmse', 'nthread':1, 'min_data_in_leaf': 2**7,
                  'bagging_fraction': 0.75, 'learning_rate': 0.03, 'objective': 'mse', 'bagging_seed': 2**7,
                  'num_leaves': 2**7, 'bagging_freq':1, 'verbose':0 }

    estimators = [sgdr]
    
    for estimator in estimators:
        print('Training Model %d: %s'%(len(preds), estimator.__class__.__name__))
        estimator.fit(train_x, train_y)
        pred_test = estimator.predict(test_x)
        preds.append(pred_test)

        # pred_train = estimator.predict(train_x)
        # print('Train RMSE for %s is %f' % (estimator.__class__.__name__, sqrt(mean_squared_error(cur_y_train, pred_train))))
        print('Test RMSE for %s is %f' % (estimator.__class__.__name__, sqrt(mean_squared_error(cur_y_test, pred_test))))

    print('Training Model %d: %s'%(len(preds), 'lightgbm'))
    estimator = lgb.train(lgb_params, lgb.Dataset(train_x, label=train_y), 300)
    pred_test = estimator.predict(test_x)
    preds.append(pred_test)
    
    # pred_train = estimator.predict(train_x)
    # print('Train RMSE for %s is %f' % ('lightgbm', sqrt(mean_squared_error(cur_y_train, pred_train))))
    print('Test RMSE for %s is %f' % ('lightgbm', sqrt(mean_squared_error(cur_y_test, pred_test))))

    print('Training Model %d: %s'%(len(preds), 'keras'))

    def baseline_model():
        model = Sequential()
        model.add(Dense(20, input_dim=train_x.shape[1], kernel_initializer='uniform', activation='softplus'))
        model.add(Dense(1, kernel_initializer='uniform', activation = 'relu'))
        
        # Compile model
        model.compile(loss='mse', optimizer='Nadam', metrics=['mse'])
        # model.compile(loss='mean_squared_error', optimizer='adam')
        return model

    estimator = KerasRegressor(build_fn=baseline_model, verbose=1, epochs=5, batch_size = 55000)
    estimator.fit(train_x, train_y)
    pred_test = estimator.predict(test_x)
    preds.append(pred_test)


    slice_end = slice_start + cur_X_test.shape[0]

    X_all_level2[ slice_start : slice_end , :] = np.c_[preds].transpose()

    slice_start = slice_end

Split train and test

In [ ]:
test_nrow = len(preds[0])
X_train_level2 = X_all_level2[ : -test_nrow, :]
X_test_level2 = X_all_level2[ -test_nrow: , :]
y_train_level2 = y_all_level2[ : -test_nrow]
y_test_level2 = y_all_level2[ -test_nrow : ]

In [ ]:
from sklearn.linear_model import LinearRegression, SGDRegressor

pred_list = {}

lr = LinearRegression()
lr.fit(X_train_level2, y_train_level2)

test_preds_lr_stacking = lr.predict(X_test_level2)
train_preds_lr_stacking = lr.predict(X_train_level2)

print('Train R-squared for %s is %f' %('train_preds_lr_stacking', sqrt(mean_squared_error(y_train_level2, train_preds_lr_stacking))))

pred_list['test_preds_lr_stacking'] = test_preds_lr_stacking

In [ ]:
sgdr= SGDRegressor(penalty = 'l2', random_state = 0 )

sgdr.fit(X_train_level2, y_train_level2)

test_preds_sgdr_stacking = sgdr.predict(X_test_level2)
train_preds_sgdr_stacking = sgdr.predict(X_train_level2)

print('Train R-squared for %s is %f' %('train_preds_lr_stacking', sqrt(mean_squared_error(y_train_level2, train_preds_sgdr_stacking))))

pred_list['test_preds_sgdr_stacking'] = test_preds_sgdr_stacking

# 4. Submision

In [ ]:
submission = sample_submission
submission_path = './'

for pred_ver in ['lr_stacking', 'sgdr_stacking']:
    print(pred_list['test_preds_' + pred_ver].clip(0,20).mean())
    
    submission['item_cnt_month'] = pred_list['test_preds_' + pred_ver].clip(0,20)
    
    submission[['ID', 'item_cnt_month']].to_csv('{0}/{1}.csv'.format(submission_path, pred_ver), index = False)